<span style="font-size:40px; font-family:'Times new roman'; font-weight:bold">
Import Data 🎒
</span>

In [8]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../src')
from binomial import binomial
from greeks import greeks

In [9]:
nvda = pd.read_csv("../data/cleanedNVDA.csv")
spy = pd.read_csv("../data/cleanedSPY.csv")

<span style="font-size:20px; font-family:'Times new roman'; font-weight:bold">
Choose All ATM Options 💻
</span>

In [19]:
nvdaSpot = nvda['spot'].iloc[0]
spySpot = spy['spot'].iloc[0]

nvda['moneyness'] = nvda['strike']/nvdaSpot
spy['moneyness'] = spy['strike']/spySpot

portfolio1 = nvda[(nvda['moneyness'] >= 0.95) & (nvda['moneyness'] <= 1.05)].copy()
portfolio1['quantity'] = 1

portfolio2 = spy[(spy['moneyness'] >= 0.95) & (spy['moneyness'] <= 1.05)].copy()
portfolio2['quantity'] = 1

In [20]:
portfolio1

,type,strike,ttm,AveragePrice,impliedVolatility,spot,dividendYield,expiry,volume,modelPrice,moneyness,quantity
5,call,90.0,0.00274,11.150,0.885133,94.309998,0.04,2025-05-02,1444.0,4.656769,0.954300,1
6,call,95.0,0.00274,7.725,0.786745,94.309998,0.04,2025-05-02,3115.0,1.233758,1.007316,1
7,call,99.0,0.00274,5.850,0.762087,94.309998,0.04,2025-05-02,576.0,0.207305,1.049730,1
55,put,90.0,0.00274,4.650,0.659183,94.309998,0.04,2025-05-02,7701.0,0.128085,0.954300,1
56,put,95.0,0.00274,6.650,0.614018,94.309998,0.04,2025-05-02,5863.0,1.588721,1.007316,1
57,put,99.0,0.00274,8.025,0.511235,94.309998,0.04,2025-05-02,442.0,4.723871,1.049730,1


<span style="font-size:20px; font-family:'Times new roman'; font-weight:bold">
Choose Top Liquid Options 💻
</span>

In [12]:
portfolio3 = nvda[nvda['volume']>0].sort_values(by='volume', ascending=False).head(10).copy()
portfolio3['quantity'] = 1

portfolio4 = spy[spy['volume']>0].sort_values(by='volume', ascending=False).head(10).copy()
portfolio4['quantity'] = 1

In [33]:
def calculateGreeks(data):
    outputGreeks = []

    for _, row in data.iterrows():
        greek = greeks(
            S=row['spot'],
            K=row['strike'],
            T=row['ttm'],
            sigma=row['impliedVolatility'],
            q=row['dividendYield'],
            ot=row['type']
        )
        greek['strike'] = row['strike']
        greek['type'] = row['type']
        greek['spot'] = row['spot']
        greek['quantity'] = row['quantity']
        outputGreeks.append(greek)

    df = pd.DataFrame(outputGreeks)
    return df

In [34]:
def calPortGreek(data, label):
    data = calculateGreeks(data)
    portfolioDelta = data['delta'].sum()
    portfolioGamma = data['gamma'].sum()
    portfolioTheta = data['theta'].sum()
    portfolioVega = data['vega'].sum()

    print(f"✅ Portfolio{label} Greeks (Unweighted):")
    print(f"Total Delta: {portfolioDelta:.2f}")
    print(f"Total Gamma: {portfolioGamma:.2f}")
    print(f"Total Theta: {portfolioTheta:.2f}")
    print(f"Total Vega: {portfolioVega:.2f}")

In [35]:
calPortGreek(portfolio1, 1)
calPortGreek(portfolio2, 2)
calPortGreek(portfolio3, 3)
calPortGreek(portfolio4, 4)

✅ Portfolio1 Greeks (Unweighted):
Total Delta: -0.22
Total Gamma: 0.51
Total Theta: 0.00
Total Vega: 7.00
✅ Portfolio2 Greeks (Unweighted):
Total Delta: -2.30
Total Gamma: 0.38
Total Theta: 0.00
Total Vega: 95.73
✅ Portfolio3 Greeks (Unweighted):
Total Delta: -2.14
Total Gamma: 0.36
Total Theta: 0.00
Total Vega: 5.53
✅ Portfolio4 Greeks (Unweighted):
Total Delta: -2.74
Total Gamma: 0.03
Total Theta: 0.00
Total Vega: 9.15


<span style="font-size:15px; font-family:'Times new roman';">
in practical, each contrect contain 100 stocks per option so i need to weight the portfolio too
</span>

In [38]:
def adjPortfolio(data, label):

    # Apply multiplier
    data = calculateGreeks(data)
    contractSize = 100
    data['delta_adj'] = data['delta'] * data['quantity'] * contractSize
    data['gamma_adj'] = data['gamma'] * data['quantity'] * contractSize
    data['theta_adj'] = data['theta'] * data['quantity'] * contractSize
    data['vega_adj']  = data['vega']  * data['quantity'] * contractSize

    print(f"✅ Portfolio{label} Greeks (Weighted):")
    print("Delta:", data['delta_adj'].sum())
    print("Gamma:", data['gamma_adj'].sum())
    print("Theta:", data['theta_adj'].sum())
    print("Vega :", data['vega_adj'].sum())

In [40]:
adjPortfolio(portfolio1, 1)
adjPortfolio(portfolio2, 2)
adjPortfolio(portfolio3, 3)
adjPortfolio(portfolio4, 4)

✅ Portfolio1 Greeks (Weighted):
Delta: -22.39346521962571
Gamma: 51.208488370000694
Theta: 0.0
Vega : 699.8057573312855
✅ Portfolio2 Greeks (Weighted):
Delta: -230.35198392961104
Gamma: 37.61370202285886
Theta: 0.0
Vega : 9573.013603576592
✅ Portfolio3 Greeks (Weighted):
Delta: -214.34861654856795
Gamma: 35.57827494886438
Theta: 0.0
Vega : 553.1594480668346
✅ Portfolio4 Greeks (Weighted):
Delta: -273.85473349377077
Gamma: 3.3493821870469676
Theta: 0.0
Vega : 914.9373089706982
